# Cabbeling simulation

Convection driven by density gradients appearing because of the mixing of
cold and hot water at the same denisty. This is cause by the nonlinearity of the equation of state of water.
We start with a stable fluid, with hot water (7.55ᵒ C) above cold water (1ᵒ C).
The fluid is initially at rest because the density is constant in the domain, but mixing of cold and hot water
generates temperatures that correspond to higher density (the maximum density of fresh water is at 4ᵒ C) and
the mixed water starts to sink.

In [1]:
using Pkg
pkg"add ClimaOcean Oceananigans#ss/for-drakkar, SeawaterPolynomials, CairoMakie, FileIO"

using Oceananigans
using Oceananigans.Models: seawater_density
using SeawaterPolynomials: TEOS10EquationOfState

     Cloning git-repo `https://github.com/CliMA/Oceananigans.jl.git`
git-repo `https://github.com/CliMA/Oceananigans.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
package versions...ng 
   Installed AxisArrays ─────────────────────── v0.4.7
   Installed JpegTurbo_jll ──────────────────── v3.1.1+0
   Installed x265_jll ───────────────────────── v3.6.0+0
   Installed libfdk_aac_jll ─────────────────── v2.0.3+0
   Installed ImageIO ────────────────────────── v0.6.9
   Installed TiffImages ─────────────────────── v0.11.2
   Installed OffsetArrays ───────────────────── v1.15.0
   Installed ClimaOcean ─────────────────────── v0.3.3
   Installed TiledIteration ─────────────────── v0.5.0
   Installed Libmount_jll ───────────────────── v2.40.3+0
   Installed HypergeometricFunctions ────────── v0.3.25
   Installed PNGFiles ───────────────────────── v0.4.3
   Installed JpegTurbo ──────────────────────── v0.1.5
   Installed LERC_jll ───────────────────────── v4.0.1+0
   Insta

We run this example on a CPU.
To run it on a GPU, replace `CPU()` with `GPU()` and crank up the resolution!

In [2]:
arch = CPU()

CPU()

We start with defining a 2D grid in the x-z plane with 512x256 grid points.
This is not enough to resolve the Kolmogorov scale so we are in an LES regime

In [3]:
Nx, Ny = 256, 256
grid = RectilinearGrid(arch,
                       size = (Nx, Ny),
                       x = (0, 0.5),
                       z = (-0.5, 0.0),
                       topology = (Bounded, Flat, Bounded))

256×1×256 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 3×0×3 halo
├── Bounded  x ∈ [0.0, 0.5]  regularly spaced with Δx=0.00195312
├── Flat y                   
└── Bounded  z ∈ [-0.5, 0.0] regularly spaced with Δz=0.00195312

We use the TEOS10 equation of state to compute the density of seawater.
This is a nonlinear equation of state that depends on temperature, salinity and pressure.
We set the salinity to zero and the gravitational acceleration to 9.81 m/s².

In [4]:
equation_of_state = TEOS10EquationOfState(reference_density=1000)
buoyancy = SeawaterBuoyancy(; equation_of_state,
                              constant_salinity=0,
                              gravitational_acceleration=9.81)

SeawaterBuoyancy{Float64}:
├── gravitational_acceleration: 9.81
├── constant_salinity: 0
└── equation_of_state: BoussinesqEquationOfState{Float64}

Since we are in an LES regime we need a LES closure. In this case we use a high-order WENO
scheme to provide us the implicit diffusion necessary to represent subgrid-scale diffusion in
the model. Other options would include an explicit closure like
the Smagorinsky model (`closure = AnisotropicMinimumDissipation()`).

In [5]:
model = NonhydrostaticModel(; grid,
                              buoyancy,
                              advection=WENO(order=7),
                              tracers=:T)

┌ Warning: Inflating model grid halo size to (4, 0, 4) and recreating grid. Note that an ImmersedBoundaryGrid requires an extra halo point in all non-flat directions compared to a non-immersed boundary grid.
└ @ Oceananigans.Models.NonhydrostaticModels ~/.julia/packages/Oceananigans/kGlF8/src/Models/NonhydrostaticModels/nonhydrostatic_model.jl:248


NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 256×1×256 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 4×0×4 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO(order=7)
├── tracers: T
├── closure: Nothing
├── buoyancy: SeawaterBuoyancy with g=9.81 and BoussinesqEquationOfState{Float64} with ĝ = NegativeZDirection()
└── coriolis: Nothing

Setting the initial conditions to an unstable equilibrium with hot water above (and below) cold water.
and some random perturbations in the velocity field to trigger the instability.
To give it a nice spin, let's initialize the temperature field with the Drakkar Ocean logo.

In [6]:
T₁, T₂ = 1, 7.55 # ᵒC
Ξᵢ = (x, z) -> 1e-4 * randn()

using FileIO

download("https://drakkar2025.sciencesconf.org/data/header/DrakkarOcean_1.png", "logo-drakkar.png")

img   = FileIO.load("logo-drakkar.png")
alpha = getproperty.(img, :alpha) .|> Float64
alpha = reverse(alpha', dims=2)
alpha = alpha[1:2:end, 1:2:end]
borderx = zeros((256 - size(alpha, 1)) ÷ 2, size(alpha, 2))
alpha   = vcat(borderx, alpha, borderx)
bordery = zeros(size(alpha, 1), (256 - size(alpha, 2)) ÷ 2)
alpha   = hcat(bordery, alpha, bordery)

Tᵢ = [ifelse(alpha[i, j] == 0, T₁, T₂) for i in 1:Nx, j in 1:Ny]

set!(model, T=Tᵢ, u=Ξᵢ, v=Ξᵢ, w=Ξᵢ)

┌ Warning: Reshaped 256×256 Matrix{Real} to set! its data to 
│ 256×1×256 Field{Center, Center, Center} on RectilinearGrid on CPU
└ @ Oceananigans.Fields ~/.julia/packages/Oceananigans/kGlF8/src/Fields/set!.jl:112


We set the time step to 0.2 seconds and the stop time to 100 seconds.

In [7]:
simulation = Simulation(model; Δt = 0.1, stop_time=100)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 100 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 1.667 minutes
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

We add a callback to print the progress of the simulation every 10 iterations.

In [8]:
function progress(sim)
    u, v, w = sim.model.velocities
    @info string("Iter: ", iteration(sim), ", time: ", prettytime(sim), ", max(w): ", maximum(abs, w))
end

add_callback!(simulation, progress, IterationInterval(10))

We use a variable time-stepping to make sure we don't overshoot the CFL condition.

In [9]:
conjure_time_step_wizard!(simulation, cfl=0.7)

Let's set an output writer to collect the density and temperature fields every second.

In [10]:
ρ = seawater_density(model)
T = model.tracers.T

output_writer = JLD2OutputWriter(model, (; ρ, T),
                                 filename = "cabbeling",
                                 schedule = TimeInterval(1),
                                 overwrite_existing = true)

simulation.output_writers[:jld2] = output_writer

JLD2OutputWriter scheduled on TimeInterval(1 second):
├── filepath: cabbeling.jld2
├── 2 outputs: (ρ, T)
├── array type: Array{Float64}
├── including: [:grid, :coriolis, :buoyancy, :closure]
├── file_splitting: NoFileSplitting
└── file size: 3.2 MiB

Running the simulation!

In [11]:
run!(simulation)

[ Info: Initializing simulation...
[ Info: Iter: 0, time: 0 seconds, max(w): 0.00037000359779137997
[ Info:     ... simulation initialization complete (15.109 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (4.993 seconds).
[ Info: Iter: 10, time: 1 second, max(w): 0.0003536769158886388
[ Info: Iter: 20, time: 2.121 seconds, max(w): 0.00038838589555555933
[ Info: Iter: 30, time: 3.399 seconds, max(w): 0.000691791700089361
[ Info: Iter: 40, time: 4.586 seconds, max(w): 0.0010588967098071553
[ Info: Iter: 50, time: 6 seconds, max(w): 0.001709777208281509
[ Info: Iter: 60, time: 7.709 seconds, max(w): 0.0025961396338666696
[ Info: Iter: 70, time: 9.390 seconds, max(w): 0.003430676502503361
[ Info: Iter: 80, time: 11.429 seconds, max(w): 0.004063951079138737
[ Info: Iter: 90, time: 13.472 seconds, max(w): 0.005129095130514524
[ Info: Iter: 100, time: 15.472 seconds, max(w): 0.0057539340193409
[ Info: Iter: 110, time: 17.416 seconds, max(w): 0.0063

## Visualizing the simulation

Let's use Makie to visualize the density and temperature fields in the x-z plane.

In [12]:
using CairoMakie

ρt = FieldTimeSeries("cabbeling.jld2", "ρ")
Tt = FieldTimeSeries("cabbeling.jld2", "T")

Nt = length(ρt)
Nx = size(ρt, 1)

i = Int(Nx / 2)
n = Observable(length(ρt.times))
ρ = @lift interior(ρt[$n], :, 1, :)
T = @lift interior(Tt[$n], :, 1, :)
x, y, z = nodes(ρt)

set_theme!(Theme(fontsize=12))
fig = Figure(size=(1000, 500))

ρrange = (minimum(ρt[1]), maximum(ρt))

axT = Axis(fig[1, 2], xlabel="x (m)", ylabel="z (m)")
xlims!(axT,  0, 0.5)
ylims!(axT, -0.5, 0)

axρ = Axis(fig[1, 3], xlabel="x (m)", ylabel="z (m)")
xlims!(axρ,  0, 0.5)
ylims!(axρ, -0.5, 0)

hm = heatmap!(axT, x, z, T, colormap=:magma, colorrange=(1.55, 7))
Colorbar(fig[1, 1], hm, label="Temperature (ᵒC)", flipaxis=false)

hm = heatmap!(axρ, x, z, ρ, colormap=Makie.Reverse(:grays), colorrange=ρrange)
Colorbar(fig[1, 4], hm, label="Density (kg m⁻³)")

record(fig, "cabbeling_2d.mp4", 1:Nt, framerate=5) do nn
    @info "Drawing frame $nn of $Nt..."
    n[] = nn
end

[ Info: Drawing frame 1 of 101...
[ Info: Drawing frame 2 of 101...
[ Info: Drawing frame 3 of 101...
[ Info: Drawing frame 4 of 101...
[ Info: Drawing frame 5 of 101...
[ Info: Drawing frame 6 of 101...
[ Info: Drawing frame 7 of 101...
[ Info: Drawing frame 8 of 101...
[ Info: Drawing frame 9 of 101...
[ Info: Drawing frame 10 of 101...
[ Info: Drawing frame 11 of 101...
[ Info: Drawing frame 12 of 101...
[ Info: Drawing frame 13 of 101...
[ Info: Drawing frame 14 of 101...
[ Info: Drawing frame 15 of 101...
[ Info: Drawing frame 16 of 101...
[ Info: Drawing frame 17 of 101...
[ Info: Drawing frame 18 of 101...
[ Info: Drawing frame 19 of 101...
[ Info: Drawing frame 20 of 101...
[ Info: Drawing frame 21 of 101...
[ Info: Drawing frame 22 of 101...
[ Info: Drawing frame 23 of 101...
[ Info: Drawing frame 24 of 101...
[ Info: Drawing frame 25 of 101...
[ Info: Drawing frame 26 of 101...
[ Info: Drawing frame 27 of 101...
[ Info: Drawing frame 28 of 101...
[ Info: Drawing frame 29 of 1

"cabbeling_2d.mp4"

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*